In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing

### 2015 ~ 2020년도 데이터 가져오기

In [2]:
filenames=['C:/workspace/p-tag/KBO_prediction_data/baseball_' + str(x) + '.csv' for x in list(range(2015,2021))]
filenames

['C:/workspace/p-tag/KBO_prediction_data/baseball_2015.csv',
 'C:/workspace/p-tag/KBO_prediction_data/baseball_2016.csv',
 'C:/workspace/p-tag/KBO_prediction_data/baseball_2017.csv',
 'C:/workspace/p-tag/KBO_prediction_data/baseball_2018.csv',
 'C:/workspace/p-tag/KBO_prediction_data/baseball_2019.csv',
 'C:/workspace/p-tag/KBO_prediction_data/baseball_2020.csv']

In [3]:
data = pd.DataFrame()
for filename in tqdm(filenames):
    temp = pd.read_csv(filename)
    data = pd.concat([data,temp])

100%|██████████| 6/6 [00:00<00:00, 13.91it/s]


In [4]:
baseball_data = data.copy()

### 날짜 데이터, HEADER_NO 삭제

In [5]:
baseball_data = baseball_data.drop(['G_ID','GDAY_DS','HEADER_NO'], axis=1)
baseball_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8656 entries, 0 to 1367
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   T_ID      8656 non-null   object 
 1   VS_T_ID   8656 non-null   object 
 2   TB_SC     8656 non-null   object 
 3   PA        8656 non-null   float64
 4   AB        8656 non-null   float64
 5   RBI       8656 non-null   float64
 6   RUN       8656 non-null   float64
 7   HIT       8656 non-null   float64
 8   H2        8656 non-null   int64  
 9   H3        8656 non-null   int64  
 10  HR        8656 non-null   float64
 11  SB        8656 non-null   int64  
 12  CS        8656 non-null   int64  
 13  SF        8656 non-null   int64  
 14  BB        8656 non-null   float64
 15  HP        8656 non-null   float64
 16  KK        8656 non-null   float64
 17  GD        8656 non-null   float64
 18  LOB       8656 non-null   float64
 19  P_HRA_RT  8656 non-null   float64
 20  P_AB_CN   8656 non-null   int6

### 무승부 제거

In [6]:
print(baseball_data['win'].value_counts())
baseball_data = baseball_data[baseball_data['win']!=0.5]
print(baseball_data['win'].value_counts())

0.0    4261
1.0    4261
0.5     134
Name: win, dtype: int64
0.0    4261
1.0    4261
Name: win, dtype: int64


### 팀 명 라벨 인코딩

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(baseball_data['T_ID'])
baseball_data.loc[:,'T_ID'] = le.transform(baseball_data['T_ID'])
baseball_data.loc[:,'VS_T_ID'] = le.transform(baseball_data['VS_T_ID'])
baseball_data.head()

,T_ID,VS_T_ID,TB_SC,PA,AB,RBI,RUN,HIT,H2,H3,...,HP,KK,GD,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,OBP,OOO,win
0,0,6,T,54.0,42.0,4.0,4.0,10.0,4,0,...,1.0,7.0,0.0,24.0,0.133333,15,2,0.352941,0.238095,0.0
1,6,0,B,46.0,40.0,5.0,5.0,8.0,4,0,...,0.0,7.0,1.0,18.0,0.090909,11,1,0.304348,0.200000,1.0
2,2,4,T,46.0,35.0,8.0,9.0,14.0,2,0,...,1.0,8.0,1.0,23.0,0.285714,14,4,0.511111,0.400000,0.0
3,4,2,B,42.0,37.0,12.0,12.0,14.0,10,0,...,1.0,8.0,1.0,17.0,0.437500,16,7,0.439024,0.378378,1.0
4,3,1,T,34.0,29.0,1.0,1.0,7.0,4,2,...,0.0,6.0,2.0,17.0,0.200000,10,2,0.333333,0.241379,0.0


### 데이터 형태가 object인 값(TB_SC)을 라벨 인코딩

In [8]:
le = LabelEncoder()
le.fit(baseball_data['TB_SC'])
baseball_data.loc[:,'TB_SC'] = le.transform(baseball_data['TB_SC'])
baseball_data.head()

,T_ID,VS_T_ID,TB_SC,PA,AB,RBI,RUN,HIT,H2,H3,...,HP,KK,GD,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,OBP,OOO,win
0,0,6,1,54.0,42.0,4.0,4.0,10.0,4,0,...,1.0,7.0,0.0,24.0,0.133333,15,2,0.352941,0.238095,0.0
1,6,0,0,46.0,40.0,5.0,5.0,8.0,4,0,...,0.0,7.0,1.0,18.0,0.090909,11,1,0.304348,0.200000,1.0
2,2,4,1,46.0,35.0,8.0,9.0,14.0,2,0,...,1.0,8.0,1.0,23.0,0.285714,14,4,0.511111,0.400000,0.0
3,4,2,0,42.0,37.0,12.0,12.0,14.0,10,0,...,1.0,8.0,1.0,17.0,0.437500,16,7,0.439024,0.378378,1.0
4,3,1,1,34.0,29.0,1.0,1.0,7.0,4,2,...,0.0,6.0,2.0,17.0,0.200000,10,2,0.333333,0.241379,0.0


### 결정 트리를 이용하여 주요 변수 알아내기
- 결정 트리의 경우 별도의 전처리가 필요 없음
- train_data1, train_target1

In [9]:
train_data1 = baseball_data[['T_ID', 'VS_T_ID', 'TB_SC', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SF', 'BB', 'HP', 'KK', 'GD', 'LOB', 'P_HRA_RT',
       'P_AB_CN', 'P_HIT_CN', 'OBP', 'OOO']]
train_target1 = baseball_data['win']

### 데이터 세트 나누기

In [10]:
from sklearn.tree import DecisionTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(train_data1, train_target1, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(6817, 24) (6817,)
(1705, 24) (1705,)


### 결정 트리 모델 학습

In [11]:
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(random_state=156)

### 결정 트리 모델 시각화

In [12]:
from sklearn.tree import export_graphviz

export_graphviz(dt_clf, out_file='tree.dot',class_names=['패','승','무'], feature_names = x_train.columns, impurity=True, filled=True)

In [ ]:
import graphviz

with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

### 피처별 중요도

In [ ]:
import seaborn as sns
import numpy as np

print('Feature importances:\n{0}'.format(np.round(dt_clf.feature_importances_,3)))

for name, value in zip(x_train.columns, dt_clf.feature_importances_):
    print('{0} : {1:.3f}'.format(name, value))

sns.barplot(x=dt_clf.feature_importances_, y = x_train.columns)

In [ ]:
importances_series = pd.Series(dt_clf.feature_importances_,x_train.columns)

In [ ]:
importances_series.sort_values(ascending=False)

### 상관 관계(corr) 활용

In [ ]:
baseball_data.info()

In [ ]:
baseball_data_corr = baseball_data.corr()

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(baseball_data_corr, cmap='Purples')

In [ ]:
baseball_data_corr['win'].sort_values(ascending=False)

### 무승부 제외 시, 주요 피처 뽑기(5개)
1. DecisionTree를 활용한 주요 피처
- RUN : 0.360827
- AB : 0.104176
- OOO: 0.060846
- PA : 0.047024
- OBP : 0.044471
2. 상관 관계(corr)를 활용한 주요 피처
- RUN : 0.559666
- PBI : 0.548687
- OBP : 0.508976
- OOO : 0.480763
- P_HIT_CN : 0.468711      
=> RUN, OOO, T_ID, VS_T_ID 으로 학습

In [ ]:
baseball_data[['RUN', 'OOO']].describe()

## 1. DecisionTree 학습

In [ ]:
baseball_data.info()

In [ ]:
DecisionTree_data = baseball_data[['T_ID','VS_T_ID','RUN','OOO']]
DecisionTree_target= baseball_data['win']

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(DecisionTree_data, DecisionTree_target, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

### GridSearchCV로 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

params={
    'max_depth':[4,8,12,16,20,24],
    'min_samples_leaf':[1,3,6,9,12,15],
    'min_samples_split':[8,16,20,24,28,32,36,40]
}

dt_clf = DecisionTreeClassifier(random_state=42)
grid_cv=GridSearchCV(dt_clf, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(train_x,train_y)

In [ ]:
print('최적 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

### 정확도 측정

In [ ]:
from sklearn.metrics import accuracy_score

dt_model = grid_cv.best_estimator_
pred = dt_model.predict(test_x)
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_y, pred)))

### 2.RandomForest 학습

In [ ]:
RandomForest_data = baseball_data[['T_ID','VS_T_ID','RUN','OOO']]
RandomForest_target= baseball_data['win']

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(RandomForest_data, RandomForest_target, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

### GridSearchCV로 하이퍼파라미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier

params={
    'max_depth':[4,8,12,16,20,24],
    'min_samples_leaf':[1,3,6,9,12,15],
    'min_samples_split':[8,16,20,24,28,32,36,40]
}

rfc = RandomForestClassifier(n_estimators=50,random_state=42,n_jobs=-1)
grid_cv=GridSearchCV(rfc, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(train_x,train_y)

### 정확도 측정

In [ ]:
from sklearn.metrics import accuracy_score

rfc_model = grid_cv.best_estimator_
pred = rfc_model.predict(test_x)
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_y, pred)))

=> RUN, OOO, T_ID, VS_T_ID 으로 학습 시, 기존(RUN,AB,PA,OOO,T_ID,VS_T_ID)보다 정확도가 떨어짐